# ASSIGNMENT 2

## Task 1

### Data Reading,Exploration and preprocessing

In [36]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms,models
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((256,256)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

path = '.data/'
train_dataset = datasets.Flowers102(root=path,split='train', transform=transform, download=True)
test_dataset = datasets.Flowers102(root=path,split='test', transform=transform, download=True)
val_dataset = datasets.Flowers102(root=path,split='val', transform=transform, download=True)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Machine learning or Deep learning model defining

In [37]:
# Define the baseline CNN model
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(32),
                                      nn.Dropout(p=0.3),

                                      nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(64),
                                      nn.Dropout(p=0.3),

                                      nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2, stride=2),
                                      nn.BatchNorm2d(128),
                                      nn.Dropout(p=0.3)
                                      )
        self.fc1 = nn.Sequential(nn.Linear(32 * 32 * 128, 512),
                                 nn.ReLU()
                                 )

        self.output = nn.Linear(512, 102)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)
        # x = x.view(x.size(0), -1)  # Flatten
        # x = self.classifier(x)
        return x

BaselineModel()

BaselineModel(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.3, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.3, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout(p=0.3, inplace=False)
  )
  (fc1): Sequential(
    (0): Linear(in_

#### Specify loss func and optimizer for base model

In [39]:
# Initialize the model, optimizer, and loss function
device = 'cuda'

print(f"Using device: {device}")
model = BaselineModel().to(device)
base_model_optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
nll_criterion = nn.NLLLoss()

Using device: cuda


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Training model func

In [ ]:
def training_model(writer:SummaryWriter,criterion,optimizer,model,epochs:int = 10):
    # Initialize TensorBoard writer
    epochs = 10
    # Train the model
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        for i, (images, labels) in enumerate(train_loader):
            # Move data to device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training loss and accuracy
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()

        # Calculate training loss and accuracy
        train_loss /= len(train_loader)
        train_accuracy = train_correct / len(train_dataset)

        # Evaluate the model on the val set
        model.eval()
        val_loss = 0
        val_correct = 0
        val_labels = []
        val_predicted = []
        with torch.no_grad():
            for images, labels in val_loader:
                # Move data to device
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_labels.extend(labels.cpu().numpy())
                val_predicted.extend(predicted.cpu().numpy())

        # Calculate test loss and accuracy
        val_loss /= len(val_loader)
        val_accuracy = val_correct / len(val_dataset)

        # Calculate F1 score
        test_f1 = f1_score(val_labels, val_predicted, average='macro')

        # Log metrics to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        writer.add_scalar('Loss/Val', val_loss, epoch)
        writer.add_scalar('Accuracy/Val', val_accuracy, epoch)
        writer.add_scalar('F1/Val', test_f1, epoch)

        # Print metrics
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {test_f1:.4f}')

    # Close TensorBoard writer
    writer.close()

#### Initialize Summary writers

#### Train base model

In [ ]:
training_model(writer=SummaryWriter(),criterion=criterion,optimizer=base_model_optimizer,model=model)

## Transfer Learning using VGG

#### Define VGG model

In [ ]:
vgg_model = models.vgg16(pretrained=True)
# for param in vgg_model.parameters():
#     param.requires_grad = False

vgg_model.classifier = nn.Sequential(
                        nn.Linear(25088, 256),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),

                        nn.Linear(256, 128),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),

                        nn.Linear(128, 32),
                        nn.LogSoftmax(dim=1))

vgg_model = vgg_model.to(device)

# Define the optimizer
vgg_optimizer = optim.Adam(vgg_model.classifier.parameters(), lr=0.001)

vgg_model

#### Train VGG

In [ ]:
training_model(writer=SummaryWriter(),criterion=criterion,optimizer=vgg_optimizer,model=vgg_model)

## Transfer Learning using ResNet-50

#### Define ResNet-50 model

In [ ]:
resnet_model = models.resnet50()
# for param in resnet_model.parameters():
#     param.requires_grad = False

resnet_model.classifier = nn.Sequential(
                        nn.Linear(512, 256),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),

                        nn.Linear(256, 128),
                        nn.ReLU(),
                        nn.Dropout(p=0.2),

                        nn.Linear(128, 32),
                        nn.LogSoftmax(dim=1))

resnet_model = resnet_model.to(device)
# Define the optimizer
resnet_optimizer = optim.Adam(resnet_model.classifier.parameters(), lr=0.001)

resnet_model

#### Train ResNet-50 model

In [ ]:
training_model(writer=SummaryWriter(),criterion=criterion,optimizer=resnet_optimizer,model=resnet_model)

### Model performance evaluation

##### Testing Model performance with test data

In [ ]:
def test_model(model, writer:SummaryWriter,criterion):
# Evaluate the model on the val set
        model.eval()
        test_loss = 0
        test_correct = 0
        test_labels = []
        test_predicted = []
        with torch.no_grad():
            for images, labels in test_loader:
                # Move data to device
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == labels).sum().item()
                test_labels.extend(labels.cpu().numpy())
                test_predicted.extend(predicted.cpu().numpy())

        # Calculate test loss and accuracy
        test_loss /= len(test_loader)
        test_accuracy = test_correct / len(test_dataset)

        # Calculate F1 score
        test_f1 = f1_score(test_labels, test_predicted, average='macro')

        # Log metrics to TensorBoard
        writer.add_scalar('Loss/Val', test_loss)
        writer.add_scalar('Accuracy/Val', test_accuracy)
        writer.add_scalar('F1/Val', test_f1)

        # Print metrics
        print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test F1: {test_f1:.4f}')

Baseline model test score

In [ ]:
test_model(model,criterion)

VGG-16 test score

In [ ]:
test_model(vgg_model,criterion)

ResNet-50 test score

In [ ]:
test_model(resnet_model,criterion)

### Conclusion and possible improvements

## Task 2

### Objective

To cluster images of flowers into distinct groups based on their visual features using unsupervised
learning techniques. The goal is to utilize a pre-trained convolutional neural network (CNN) to
extract features from the images and then apply a clustering algorithm to categorize the flowers

### Feature Extraction

In [ ]:
# Extract features from the flower images
features = []
for images, _ in train_loader:
    images = images.to(device)
    feature_vectors = resnet_model(images)
    features.append(feature_vectors.detach().cpu().numpy())

features = np.concatenate(features, axis=0)

### Clustering: Cluster the extracted features using K-means.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Perform K-means clustering on the extracted features
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(features)

# Reduce the feature dimensionality using PCA for visualization
pca = PCA(n_components=2)
pca_features = pca.fit_transform(features)

### Clusters visualization

In [ ]:
import matplotlib.pyplot as plt

# Plot the clustered data points
plt.figure(figsize=(8, 6))
plt.scatter(pca_features[:, 0], pca_features[:, 1], c=cluster_labels, cmap='viridis')
plt.title('Flower Image Clusters')
plt.xlabel('PCA Dimension 1')
plt.ylabel('PCA Dimension 2')
plt.show()